# Capstone Project - Gastronomy Tourism Appliaction
### Applied Data Science Capstone by IBM/Coursera

# Gastronomy Tourism Appliaction

## Introduction: Business Problem <a name="introduction"></a> 
What is Gastronomy Tourism??

Food is now a main motivation for travelers choosing their destinations. Travelers are spending more time and money on unique food and beverage experiences. We have seen a global increase in food tour companies, food and beverage focused events and food and beverage experience focused marketing efforts. <br> <br> But can we use machine learinig to help improve the gastronomy tourism entusiasts experiance? <br> I am a burger enutuziast, and i am intrested in visiting some nice buger specific venues and also have a nice trip in some of Europes capitals. Can we come up with a data science aproach that can help me answer my question: ***in wich cities can i find the best burger joints?*** A nice traveling location in my case should have lots of cheap and high rated burger venues and also should be as close as possible to my hometown Bucharest. <br> <br> By developing a data science framework to answer my question, maybe we can further help other gastronomy tourists whith different intrests (lets say pizza lovers) find theyr next vacation location. <br> <br> **Methodological approach**: <br> <br> - Phase1: I will scrap 'http://techslides.com/list-of-countries-and-capitals' to get the latitude and longitude coordinates for all Europe's capital cities <br> <br> - Phase2: Next, I will get the venues near the capital city centers of Europe <br> <br> - Phase3:  Extract other relevent info (likes count, photos count , price cateqory, rating etc.) using the forsquae app <br> <br> - Phase4:  I will prepare the data for the clustering phase <br> <br> - Phase5: Clustering the citties using the venu info we extracte from foresquare <br> <br> - Phase 6: Map vizualization usnig Folium


Instaling Geopy for distance calcukation and Folium for the map plots

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

!conda install -c conda-forge geopy
print('Geopy installed and imported!')

Importing relevant library

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium

# Importing the geodesic module from the library 
from geopy.distance import great_circle

print("Libraries imported.")

Libraries imported.


# Phase 1: Data Scraping

I will data scrap 'http://techslides.com/list-of-countries-and-capitals' to get the latitude and longitude coordinates for all Europe's capital cities.
I will use BeautifulSoup to scrap the data and put it in a padas data frame.

In [4]:
data = requests.get('http://techslides.com/list-of-countries-and-capitals').text
soup = BeautifulSoup(data, 'html.parser')

Country_Name = []
Capital_Name = [] 
Capital_Latitude = [] 
Capital_Longitude = [] 
Country_Code = [] 
Continent_Name = []


for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(cells[0].text != 'Country Name'):
        Country_Name.append(cells[0].text)
        Capital_Name.append(cells[1].text)
        Capital_Latitude.append(cells[2].text)
        Capital_Longitude.append(cells[3].text)
        Country_Code.append(cells[4].text)
        Continent_Name.append(cells[5].text)

In [5]:
europe_df = pd.DataFrame({"Country_Name": Country_Name,
                           "Capital_Name": Capital_Name,
                           "Capital_Latitude": Capital_Latitude,
                           "Capital_Longitude": Capital_Longitude,
                           "Country_Code": Country_Code,
                           "Continent_Name": Continent_Name})

europe_df = europe_df[europe_df['Continent_Name'] == 'Europe'].reset_index(drop=True)
europe_df.head(10)

,Country_Name,Capital_Name,Capital_Latitude,Capital_Longitude,Country_Code,Continent_Name
0,Aland Islands,Mariehamn,60.116667,19.9,AX,Europe
1,Albania,Tirana,41.31666667,19.816667,AL,Europe
2,Andorra,Andorra la Vella,42.5,1.516667,AD,Europe
3,Armenia,Yerevan,40.16666667,44.5,AM,Europe
4,Austria,Vienna,48.2,16.366667,AT,Europe
5,Azerbaijan,Baku,40.38333333,49.866667,AZ,Europe
6,Belarus,Minsk,53.9,27.566667,BY,Europe
7,Belgium,Brussels,50.83333333,4.333333,BE,Europe
8,Bosnia and Herzegovina,Sarajevo,43.86666667,18.416667,BA,Europe
9,Bulgaria,Sofia,42.68333333,23.316667,BG,Europe


# Phase 2: Getting Venue Data 

Next, I want to get the venues near the capital city centers of Europe. I am interested in the venues that are in a radius of 500m form the city centers and also serves burgers.
To get the venues i will use the forsquare app. 

In [6]:
CLIENT_ID = '33LRYQTSCFQRDJQP4YDDTQGXLZWS0XQL4HPC4EOMRBVYG2RL' # your Foursquare ID
CLIENT_SECRET = 'TYHUSJA10OJS1AKEIHE3TJYEOQT4O1GP5EK4G2BS4FUWHGVC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 33LRYQTSCFQRDJQP4YDDTQGXLZWS0XQL4HPC4EOMRBVYG2RL
CLIENT_SECRET:TYHUSJA10OJS1AKEIHE3TJYEOQT4O1GP5EK4G2BS4FUWHGVC


In [7]:

for i in range(0, len(europe_df)):
    latitude = europe_df.iloc[i,2]
    longitude = europe_df.iloc[i,3]
    #print(latitude, longitude)
    
    search_query = 'Burger'
    radius = 1000
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    url
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)

    if i==0:
        df_full = dataframe
    else:
        df_full = df_full.append(dataframe, ignore_index=True, sort=True)
df_full

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,50cb1d56e4b09d952fee57e5,NaN,AL,NaN,Shqipëria,NaN,603,[Shqipëria],"[{'label': 'display', 'lat': 41.32171190875343...",41.321712,19.814021,NaN,NaN,NaN,Urban Burger House,v-1572546931,NaN
1,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,56dd72ba498ec09ad59cdae3,NaN,AM,Երևան,Հայաստան,NaN,1269,"[Երևան, Yerevan, Հայաստան]","[{'label': 'display', 'lat': 40.171468, 'lng':...",40.171468,44.513532,NaN,NaN,Yerevan,"""Classic Burger"" Rossia Mall",v-1572546931,NaN
2,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4b5dece2f964a520237429e3,Annagasse 3a,AT,Wien,Österreich,NaN,616,"[Annagasse 3a, 1010 Wien, Österreich]","[{'label': 'display', 'lat': 48.20452995266250...",48.204530,16.371440,NaN,1010,Wien,Burger King,v-1572546931,NaN
3,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,53f99c0f498eacc7b117f44c,Getreidemarkt 11,AT,Wien,Österreich,NaN,342,"[Getreidemarkt 11, 1060 Wien, Österreich]","[{'label': 'display', 'lat': 48.20169696038045...",48.201697,16.362811,NaN,1060,Wien,Addicted to Rock Bar & Burger,v-1572546931,93461172
4,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,4b196761f964a520c6dc23e3,Burggasse 12,AT,Wien,Österreich,NaN,1012,"[Burggasse 12, 1070 Wien, Österreich]","[{'label': 'display', 'lat': 48.20417714350081...",48.204177,16.354541,NaN,1070,Wien,Die Burgermacher,v-1572546931,NaN
5,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,56e7c022498ed7e21f3eb69a,"пл. Свободы, 17",BY,Мінск,Беларусь,NaN,1029,"[пл. Свободы, 17, Мінск, Беларусь]","[{'label': 'display', 'lat': 53.90477732804782...",53.904777,27.553224,NaN,NaN,Мінск,Burger King,v-1572546932,NaN
6,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,5d08fad78c9f41002dbe251e,"Немига, 5",BY,Мінск,Беларусь,NaN,1111,"[Немига, 5, Мінск, Беларусь]","[{'label': 'display', 'lat': 53.902794, 'lng':...",53.902794,27.550389,NaN,NaN,Мінск,Black Star Burger,v-1572546932,NaN
7,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,536b95d4498e42ca56144933,NaN,BY,Мінск,Беларусь,NaN,690,"[Мінск, Беларусь]","[{'label': 'display', 'lat': 53.90546965198043...",53.905470,27.561701,NaN,NaN,Мінск,Burger box,v-1572546932,NaN
8,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,5d397a3179df8a00088003e7,NaN,BY,Мінск,Беларусь,NaN,869,"[Мінск, Беларусь]","[{'label': 'display', 'lat': 53.89512, 'lng': ...",53.895120,27.556313,NaN,NaN,Мінск,The Burger,v-1572546932,NaN
9,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,58012479d67c9dfc28da3611,Октябрьская плошадь,BY,Мінск,Беларусь,NaN,1169,"[Октябрьская плошадь, Мінск, Беларусь]","[{'label': 'display', 'lat': 53.909454, 'lng':...",53.909454,27.574432,NaN,NaN,Мінск,Вега Бургер Vega Burger,v-1572546932,NaN


# Phase 3: Explore a Given Venue

For every 300 burger venues I found I will extract other relevent info (likes count, photos count , price cateqory, rating etc.) using the forsquae app. Also I will calculate the distande from Bucharest (my residential city) to city where the burger venues are located using Geopy library.

In [8]:
for i in range(0, len(df_full)):
    venue_id = df_full.loc[i,'id'] 
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    url

    result = requests.get(url).json()
    print(result['response']['venue'].keys())
    venues = result['response']['venue']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    dataframe = dataframe.loc[:, ['dislike','id','likes.count', 'photos.count', 'price.message', 'rating']]
    
    if i==0:
        df_full2 = dataframe
    else:
        df_full2 = df_full2.append(dataframe, ignore_index=True, sort=True)
        
df_full2.head()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])
dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])
dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'description', 'page', 'hereNow', 'createdAt'

,dislike,id,likes.count,photos.count,price.message,rating
0,False,50cb1d56e4b09d952fee57e5,0,1,Cheap,NaN
1,False,56dd72ba498ec09ad59cdae3,1,1,Cheap,NaN
2,False,4b5dece2f964a520237429e3,139,123,Moderate,5.6
3,False,53f99c0f498eacc7b117f44c,76,75,Moderate,7.7
4,False,4b196761f964a520c6dc23e3,114,72,Expensive,7.2


In [9]:
df_input = pd.merge(df_full, df_full2, how='left', on =['id', 'id'])
df_input.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id,dislike,likes.count,photos.count,price.message,rating
0,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,50cb1d56e4b09d952fee57e5,NaN,AL,NaN,Shqipëria,NaN,603,[Shqipëria],"[{'label': 'display', 'lat': 41.32171190875343...",41.321712,19.814021,NaN,NaN,NaN,Urban Burger House,v-1572546931,NaN,False,0,1,Cheap,NaN
1,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,56dd72ba498ec09ad59cdae3,NaN,AM,Երևան,Հայաստան,NaN,1269,"[Երևան, Yerevan, Հայաստան]","[{'label': 'display', 'lat': 40.171468, 'lng':...",40.171468,44.513532,NaN,NaN,Yerevan,"""Classic Burger"" Rossia Mall",v-1572546931,NaN,False,1,1,Cheap,NaN
2,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4b5dece2f964a520237429e3,Annagasse 3a,AT,Wien,Österreich,NaN,616,"[Annagasse 3a, 1010 Wien, Österreich]","[{'label': 'display', 'lat': 48.20452995266250...",48.204530,16.371440,NaN,1010,Wien,Burger King,v-1572546931,NaN,False,139,123,Moderate,5.6
3,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,53f99c0f498eacc7b117f44c,Getreidemarkt 11,AT,Wien,Österreich,NaN,342,"[Getreidemarkt 11, 1060 Wien, Österreich]","[{'label': 'display', 'lat': 48.20169696038045...",48.201697,16.362811,NaN,1060,Wien,Addicted to Rock Bar & Burger,v-1572546931,93461172,False,76,75,Moderate,7.7
4,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,4b196761f964a520c6dc23e3,Burggasse 12,AT,Wien,Österreich,NaN,1012,"[Burggasse 12, 1070 Wien, Österreich]","[{'label': 'display', 'lat': 48.20417714350081...",48.204177,16.354541,NaN,1070,Wien,Die Burgermacher,v-1572546931,NaN,False,114,72,Expensive,7.2


In [10]:
# Loading the lat-long data for Bucharest
bucharest = (44.43333333, 26.1) 

for i in range(0,len(df_input)):
    df_input.loc[i,'Distance_from_Buc'] = great_circle((df_input.loc[i,'location.lat'], df_input.loc[i,'location.lng']), (bucharest)).km
df_input.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id,dislike,likes.count,photos.count,price.message,rating,Distance_from_Buc
0,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,50cb1d56e4b09d952fee57e5,NaN,AL,NaN,Shqipëria,NaN,603,[Shqipëria],"[{'label': 'display', 'lat': 41.32171190875343...",41.321712,19.814021,NaN,NaN,NaN,Urban Burger House,v-1572546931,NaN,False,0,1,Cheap,NaN,617.836519
1,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,56dd72ba498ec09ad59cdae3,NaN,AM,Երևան,Հայաստան,NaN,1269,"[Երևան, Yerevan, Հայաստան]","[{'label': 'display', 'lat': 40.171468, 'lng':...",40.171468,44.513532,NaN,NaN,Yerevan,"""Classic Burger"" Rossia Mall",v-1572546931,NaN,False,1,1,Cheap,NaN,1582.757537
2,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",False,4b5dece2f964a520237429e3,Annagasse 3a,AT,Wien,Österreich,NaN,616,"[Annagasse 3a, 1010 Wien, Österreich]","[{'label': 'display', 'lat': 48.20452995266250...",48.204530,16.371440,NaN,1010,Wien,Burger King,v-1572546931,NaN,False,139,123,Moderate,5.6,855.839283
3,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,53f99c0f498eacc7b117f44c,Getreidemarkt 11,AT,Wien,Österreich,NaN,342,"[Getreidemarkt 11, 1060 Wien, Österreich]","[{'label': 'display', 'lat': 48.20169696038045...",48.201697,16.362811,NaN,1060,Wien,Addicted to Rock Bar & Burger,v-1572546931,93461172,False,76,75,Moderate,7.7,856.278957
4,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,4b196761f964a520c6dc23e3,Burggasse 12,AT,Wien,Österreich,NaN,1012,"[Burggasse 12, 1070 Wien, Österreich]","[{'label': 'display', 'lat': 48.20417714350081...",48.204177,16.354541,NaN,1070,Wien,Die Burgermacher,v-1572546931,NaN,False,114,72,Expensive,7.2,856.950834


# Phase 4: Data Preprocesing

Next I will prepare the data for the clustering phase. First I will drop Romania my home country from the analysis. Next I encode the price and fill the missing values. The 'Distance_from_Buc' is transformed as one over 'Distance_from_Buc', this is done so that all varaibles have same inerpretability. 
Next the venue data is agregated at country level by taking the mean of the imput values an the values are standardised.

In [11]:
df_input = df_input[df_input['location.cc'] != 'RO']
df_input = df_input[df_input['location.cc'] != 'RO']

df_input.loc[df_input['price.message'] == 'Cheap', 'price_cat'] = 3
df_input.loc[df_input['price.message'] == 'Moderate', 'price_cat'] = 2
df_input.loc[df_input['price.message'] == 'Expensive', 'price_cat'] = 1
df_input['rating'] = df_input['rating'].fillna(0)
df_input['price_cat'] = df_input['price_cat'].fillna(1)
df_input['Distance_from_Buc_tr'] = 1/df_input['Distance_from_Buc']

In [12]:
df_input_group = df_input.loc[:, ['location.cc', 'likes.count', 'photos.count', 'price_cat', 'rating', 'Distance_from_Buc_tr']].groupby('location.cc').mean()

df_input_group.head()

,likes.count,photos.count,price_cat,rating,Distance_from_Buc_tr
location.cc,,,,,
AL,0.000000,1.0,3.000000,0.000000,0.001619
AM,1.000000,1.0,3.000000,0.000000,0.000632
AT,109.666667,90.0,1.666667,6.833333,0.001168
BE,4.600000,3.2,3.000000,1.140000,0.000565
BG,211.000000,222.5,2.000000,3.625000,0.003374


In [13]:
from sklearn import preprocessing

X = df_input_group.loc[:, ['likes.count', 'photos.count', 'price_cat', 'rating', 'Distance_from_Buc_tr']]

X = preprocessing.StandardScaler().fit(X).transform(X.astype('float'))
X[0:5]


array([[-0.52648809, -0.52760714,  0.75101123, -1.31280629,  0.88578277],
       [-0.51694283, -0.52760714,  0.75101123, -1.31280629, -0.56238912],
       [ 0.5203089 ,  0.56871959, -3.24275938,  1.94723846,  0.22415702],
       [-0.48257989, -0.50050692,  0.75101123, -0.76893541, -0.66071065],
       [ 1.48756204,  2.20089141, -2.24431673,  0.41660769,  3.46186402]])

# Phaese 5: Clustering

Next, I will run K-means clustering algorithm on the following variables: 'likes.count', 'photos.count', 'price_cat', 'rating', 'Distance_from_Buc_tr'.
I want to find the cluster of capital citys with the best burger venus (that have most likes, high ratings, high number of photos, cheap prices and also be relatively close to my hometown Bucharest).

In [14]:
from sklearn.cluster import KMeans 

clusterNum = 4
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[1 1 0 1 0 3 3 3 0 3 1 3 3 3 3 3 1 1 1 1 3 3 1 1 1 1 1 1 3 3 3 3 3 3 3 3 0
 2 1]


The countrys are grouped in 4 cluster and the assignment of each country to it's coresponding cluster si listed below:

In [15]:
df_input_group['labels'] = labels
df_input_group

,likes.count,photos.count,price_cat,rating,Distance_from_Buc_tr,labels
location.cc,,,,,,
AL,0.000000,1.000000,3.000000,0.000000,0.001619,1
AM,1.000000,1.000000,3.000000,0.000000,0.000632,1
AT,109.666667,90.000000,1.666667,6.833333,0.001168,0
BE,4.600000,3.200000,3.000000,1.140000,0.000565,1
BG,211.000000,222.500000,2.000000,3.625000,0.003374,0
BY,17.333333,35.833333,2.666667,3.983333,0.000945,3
CY,73.166667,79.583333,2.416667,3.350000,0.000833,3
CZ,99.000000,50.500000,3.000000,4.400000,0.000928,3
DE,181.000000,63.500000,2.214286,3.985714,0.000773,0


Looking at some agregated results by cluster, it seems that the clusters with the best burger venues are "0" and "2". So, the most atractive cities for a burger frenzy are Kiev ("UA" Ucrain in cluster "2"), Viena ("AT" Austria in cluster "0"), Sofia ("BG" Bulgaria in cluster"0"), Berlin ('DE" Germany in cluster "0") and Ankara ("TR" Turkey in cluster "0"). Well acording to wikipedia Turkey, by land mass, is positioned 95% in Asia and 5% in Europe, so maybe that is the reason that forsquare app lables Turkey as a Europe country.

In [16]:
df_input_group_mean = df_input_group.groupby('labels').mean()
df_input_group_mean.head()

,likes.count,photos.count,price_cat,rating,Distance_from_Buc_tr
labels,,,,,
0,127.666667,97.625000,1.954613,4.192262,0.001662
1,3.705470,6.002393,2.960342,0.417333,0.001183
2,617.142857,461.142857,2.714286,4.114286,0.001341
3,50.933258,40.407351,2.751777,4.219742,0.000729


Now let's take a look at the burger venues we found in the most atractive cities.

In [17]:
country_list = df_input_group[df_input_group['labels'].isin([0,2])].index.tolist()
map_input = df_input[df_input['location.cc'].isin(country_list)]
map_input = map_input.loc[:,['location.cc', 'location.lat', 'location.lng', 'name']]
map_input
 

,location.cc,location.lat,location.lng,name
2,AT,48.204530,16.371440,Burger King
3,AT,48.201697,16.362811,Addicted to Rock Bar & Burger
4,AT,48.204177,16.354541,Die Burgermacher
16,BG,42.686165,23.317540,BURGER STATION
17,BG,42.683256,23.314404,Burger Station
18,BG,42.693412,23.319779,Boom! Burgers
19,BG,42.693489,23.319630,Boom! Burgers HQ
62,DE,52.522346,13.412718,Burger King
63,DE,52.527270,13.398797,Shiso Burger
64,DE,52.520444,13.387446,Burger King


# Phase 6: Vizualization

Finaly, I will pin point on the map using Folium the burger venues we have discovered.

In [18]:
bucharest = (44.43333333, 26.1) 

map_toronto = folium.Map(location=[44.43333333, 26.1], zoom_start=3)

# add markers to map
for lat, lng, location, name in zip(map_input['location.lat'], map_input['location.lng'], map_input['location.cc'], map_input['name']):
    label = '{}, {}'.format(location, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

# Thats all Colegues, many thanks for your attention!